# Scott Breitbach
## 31-May-2021
## DSC550, Milestone 5: Final Project

# Case Study: Beans

## Introduction

My initial intention with this project was to utilize a data set from a project I’ve been working on in my job in the hopes of getting not just a better understanding of the process, but hopefully gaining some insights into how the process could be improved. Unfortunately, I quickly realized that the data set I was using was maybe not ideal for applying the topics covered in this class. There is lots of missing data and by the time the useful data gets sorted out, there really wasn’t enough left to work with.  

Following this minor setback, I was able to find another data set about dry beans that was not just a better fit for this class, but also appealed to some of my interests. I grew up farming and considered going into plant genetics before falling into the field of microbiology.  So a data set about a food crop felt like a good option with which to proceed.


## Business Problem / Data

Dry beans are an important food crop worldwide. There is a lot of diversity in bean varieties and both classification and seed quality are important factors in the agricultural industry for maximizing production in order to feed the growing world population.  

This data set was created utilizing computer vision to automatically generate 16 features based on high resolution images of 13,611 individuals from 7 different kinds of beans. These features are based on various measurements of the beans on 12 form dimensions (such as area, length, and perimeter) and 4 shape factors.  

With an effective method to classify beans, the agricultural needs of bean uniformity can be met, both in regards to sorting beans as well as quality and consistency within bean varieties.  

Data source: https://archive.ics.uci.edu/ml/datasets/Dry+Bean+Dataset

In [1]:
# Load Libraries
import pandas as pd
import numpy as np

# Set random seed
np.random.seed(42)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


### Load Data

In [2]:
# Load data to a DataFrame
beanDF = pd.read_excel('DryBeanDataset/Dry_Bean_Dataset.xlsx')

In [3]:
# Get a look at the data
beanDF.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395,610.291,208.178117,173.888747,1.197191,0.549812,28715,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,SEKER
1,28734,638.018,200.524796,182.734419,1.097356,0.411785,29172,191.272750,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430,SEKER
2,29380,624.110,212.826130,175.931143,1.209713,0.562727,29690,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,SEKER
3,30008,645.884,210.557999,182.516516,1.153638,0.498616,30724,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199,SEKER
4,30140,620.134,201.847882,190.279279,1.060798,0.333680,30417,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166,SEKER


The values for different features vary by orders of magnitude, so these will need to be scaled prior to modeling. 

In [4]:
# Look at the shape of the DataFrame
beanDF.shape

(13611, 17)

In [5]:
# See the relative numbers of each target category
beanDF.value_counts('Class')

Class
DERMASON    3546
SIRA        2636
SEKER       2027
HOROZ       1928
CALI        1630
BARBUNYA    1322
BOMBAY       522
dtype: int64

We can see here that an immediate improvement to the data set would be to increase the number of data points for beans like 'CALI', 'BARBUNYA', and particularly 'BOMBAY', or better yet make the entire data set more uniform by increasing the data points for each bean to around 3,500 in order to match 'DERMASON'. 

## Graphical Analysis

## Dimensionality & Feature Reduction / Engineering

## Model Selection & Evaluation

## Conclusion